In [ ]:
"""
CNN
This script is used to visualize that why my network decide this image as this class

"""

In [ ]:
"""
Decide gpu usage
"""
%env CUDA_VISIBLE_DEVICES=3
this_gpu_use_fraction = 0.5

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = this_gpu_use_fraction
set_session(tf.Session(config = config))


In [ ]:
"""
import basic libraries and NN-libraries
"""
import numpy as np
import os
import re
import time
import random
import cv2
import scipy.misc

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from math import ceil
import pandas as pd
from PIL import Image

import keras
from keras.models import load_model, Model
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import keras.backend as K

K.set_learning_phase(0)

In [ ]:
def op_img(x, crop_size, output_size):
        # crop size: w, h
        # output size, w, h, c
        crop_w, crop_h = crop_size
        out_w, out_h, out_c = output_size
        X_ = np.array([ np.array(img_center_crop(Image.open(i), 
                                                      target_size=(crop_w, crop_h)), dtype='float32') for i in x] )
        X_ = np.array([scipy.misc.imresize(i, size=(out_w, out_h)) for i in X_]) # add this line to resize images
        X_ = X_.reshape((len(X_), out_w, out_h, out_c))
        return X_
    
def img_center_crop(img, target_size):
    # return center cropprd image (not resizing)
    # img should be a PIL image object
    # target size should be a tuple, eg (224, 224)
    width, height = img.size
    if width <= target_size[0] and height <= target_size[1]:
        return img
    left = (width - target_size[0])/2
    right = (width + target_size[0])/2
    top = (height - target_size[1])/2
    bottom = (height + target_size[1])/2
    
    new_img = img.crop((left, top, right, bottom))
    return new_img

def transparent_cmap(cmap, N=255):
    "Copy colormap and set alpha values"
    mycmap = cmap
    mycmap._init()
    mycmap._lut[:,-1] = np.linspace(0, 0.8, N+4)
    return mycmap

mycmap = transparent_cmap(plt.cm.rainbow)

In [ ]:
K.clear_session()
this_model = "/home/seanyu/project/CCP/model/model_hard_negative_thresh10_gen2_k3.h5"

model = load_model(this_model)
#model = model.get_layer('model_1') # if you use multiple gpu to train

model.summary()

relu_out = Model(inputs= [model.input], outputs = [model.output, model.get_layer('activation_40').output])


In [ ]:
outcome_csv = "/home/seanyu/project/CCP/res_csv/testing_hard_negative_thresh10_gen2_k3.csv"
test = pd.read_csv(outcome_csv)
test.head()

In [ ]:
"""
Get false positive and false negative samples
"""
posi_thres = 0.95
nega_thres = 0.5

fp_cases = test[(test.y_true == 0) & (test.y_pred > posi_thres)]
fn_cases = test[(test.y_true == 1) & (test.y_pred <= nega_thres)]
tp_cases = test[(test.y_true == 1) & (test.y_pred > nega_thres)]
tn_cases = test[(test.y_true == 0) & (test.y_pred < posi_thres)]

print('numbers of false positve samples under thresold %g : %g' % (posi_thres, len(fp_cases) ))
print('numbers of false negative samples under thresold %g : %g' % (nega_thres, len(fn_cases) ))
print('numbers of true positve samples under thresold %g : %g' % (nega_thres, len(tp_cases) ))
print('numbers of true negative samples under thresold %g : %g' % (posi_thres, len(tn_cases) ))

# manually reorder 
fp_cases = fp_cases.sort_values(by = 'y_pred', ascending= False)
fn_cases = fn_cases.sort_values(by = 'y_pred', ascending= True)
tp_cases = tp_cases.sort_values(by = 'y_pred', ascending= False)
tn_cases = tn_cases.sort_values(by = 'y_pred', ascending= True)

In [ ]:
def feed_in_plot(df_to_plot, nrow_to_go, force_b = True, to_save = False):
    # df_to_plot: a data frame (for example: fp_cases / fn_cases)
    # nrow_to_go: how many cases to go
    # force_b: force_binary?
    # to_save: save picture (under current dir, create a folder named: heatmap_plotting)
    # return: array to 
    
    #df_to_plot = df_to_plot.sort_values(by = 'y_pred', ascending = False)
    df_to_plot = df_to_plot.reset_index(drop = True)
    
    total_len = len(df_to_plot)
    arr = []
    # define input size / final size
    w, h = 200, 200
    y, x = np.mgrid[0:h, 0:w]
    
    for i in np.arange(len(df_to_plot)):
        this_line = df_to_plot.iloc[i] # get a line in the df
        
        im_to_go = [this_line.png_name]
        this_im = op_img(im_to_go, (100, 100), (w, h, 3))
        im_ori = this_im.copy()
        this_im = this_im.astype('float32')
        this_im = preprocess_input(this_im)
        result = relu_out.predict(this_im)
        
        output = result[0] # last layer
        filters = result[1] # filter layer
        
        if force_b:
            # force it use the true label case to plot it
            output = np.array([0, 0])
            output[int(this_line.y_true)] += 1
            #output = output.reshape(2,1)
        else:
            output = output.swapaxes(0, 1)

        gap_w = np.dot(model.get_layer('output').get_weights()[0], output)
        print(gap_w.shape)
        
        gap_w = gap_w.reshape(1, 1, 1, filters.shape[-1])
        heat_map = gap_w * filters
        heat_map = heat_map.sum(axis = 3).reshape(filters.shape[1], filters.shape[2])
        heat_map = np.array(Image.fromarray(heat_map.astype('uint8')).resize((200,200), Image.CUBIC))
        
        im_ori = im_ori.reshape(w, h,3)
        heat_map = heat_map.astype('float32')
        arr.append(im_ori)
        arr.append(heat_map)
        
        title_text = 'real label: ' + str(int(this_line.y_true)) + ' / Prob(1): ' + str(float("{0: .4f}".format(this_line.y_pred) ))
        fig, ax = plt.subplots(nrows=1, ncols= 1)
        #fig, ax = plt.subplots(nrows=1, ncols= 2, sharex= True, sharey=True)
        #ax.imshow(im_ori)
        
        ax.imshow(im_ori)
        cb = ax.contourf(x, y, heat_map, 5, cmap=mycmap, alpha = 0.15)
        ax.set_title(title_text)
        ax.axis('off')
        plt.show()
        
        if i >= (nrow_to_go - 1):
            break
        
    return arr

In [ ]:
my_arr = feed_in_plot(fp_cases, nrow_to_go=10, force_b=False)

In [ ]:
def img_combine(img,ncols=5,size=1,path=False):
    nimg=len(img)
    nrows=int(ceil(nimg/ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey=True, figsize=(ncols*size,nrows*size))

    if nrows==0:
        return
    elif ncols == 1:
        for r, ax in zip(np.arange(nrows), axes):
            nth=r
            if nth < nimg:
                ax.imshow(img[nth],cmap='rainbow')
            ax.set_axis_off()
    elif nrows==1:
        for c, ax in zip(np.arange(ncols), axes):
            nth=c
            if nth < nimg:
                ax.imshow(img[nth],cmap='rainbow' )
            ax.set_axis_off()
    else:
        for r, row in zip(np.arange(nrows), axes):
            for c, ax in zip(np.arange(ncols), row):
                nth=r*ncols+c
                if nth < nimg:
                    ax.imshow(img[nth],cmap='rainbow')
                ax.set_axis_off()
    
    if path:
        plt.savefig(path, dpi = 300)
    plt.show()

In [ ]:
img_combine(my_arr, ncols=4, size=4, path=False)

In [ ]:
force_binary = [1, 0] # [] /  [0, 1] / [1, 0]

im_to_go = [test.iloc[2].png_name]
this_im = op_img(im_to_go, (100, 100), (200, 200, 3))
im_ori = this_im.copy()
this_im = this_im.astype('float32')
this_im = preprocess_input(this_im)
result = relu_out.predict(this_im)

output = result[0] # last layer
filters = result[1] # filter layer


if force_binary:
    output = np.array(force_binary)
else:
    output = output.swapaxes(0, 1)

gap_w = np.dot(model.get_layer('output').get_weights()[0], output)
print(gap_w.shape)

gap_w = gap_w.reshape(1, 1, 1, filters.shape[-1])
heat_map = gap_w * filters
heat_map = heat_map.sum(axis = 3).reshape(filters.shape[1], filters.shape[2])
heat_map = np.array(Image.fromarray(heat_map.astype('uint8')).resize((200,200), Image.CUBIC))


im_ori = im_ori.reshape(200, 200,3)
heat_map = heat_map.astype('float')

w, h = 200, 200
y, x = np.mgrid[0:h, 0:w]

fig, ax = plt.subplots(1, 1)
ax.imshow(im_ori)
cb = ax.contourf(x, y, heat_map, 5, cmap=mycmap, alpha = 0.15)
ax.axis('off')
#plt.colorbar(cb)
plt.show()

In [ ]:
im_ori.size